In [1]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time
from shapedetector import ShapeDetector

In [2]:
vs = cv2.VideoCapture('ball2_slow.mp4')
#time.sleep(2.0)

In [3]:
# def onMouse(event, x, y, flags, param):

#     if event == cv2.EVENT_LBUTTONDOWN:
#         # draw circle here (etc...)
#         print(x, y)
#         return(x, y)

# frame = vs.read()[1]
# cv2.namedWindow('image', cv2.WINDOW_NORMAL)
# cv2.imshow('image',frame)
# location_of_basket = cv2.setMouseCallback('image', onMouse)
# print(location_of_basket)

In [4]:
position_of_basket = (530, 209)
radius_of_basket = 22

In [5]:
radii = []
recently_scored = 0
while True:
    image = vs.read()[1]
    if image is None:
        break
    resized = imutils.resize(image, width=300)
    ratio = image.shape[0] / float(resized.shape[0])
 
    # convert the resized image to grayscale, blur it slightly,
    # and threshold it
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.bitwise_not(thresh)
    cv2.imshow('thresh', thresh)
    # find contours in the thresholded image and initialize the
    # shape detector
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    sd = ShapeDetector()
    # loop over the contours

    for c in cnts:
        # compute the center of the contour, then detect the name of the
        # shape using only the contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int((M["m10"] / M["m00"]) * ratio)
            cY = int((M["m01"] / M["m00"]) * ratio)
            shape = sd.detect(c)
            
            if shape != 'circle':
                continue
            
            
            #print(cv2.contourArea(c))
            # multiply the contour (x, y)-coordinates by the resize ratio,
            # then draw the contours and the name of the shape on the image
            c = c.astype("float")
            c *= ratio
            c = c.astype("int")
            
            (x,y),radius = cv2.minEnclosingCircle(c)
            center = (int(x),int(y))
            radius = int(radius)
            if radius>100 or radius < 7 :
                continue
            radii.append(radius)
            
            try:
                ellipse = cv2.fitEllipse(c)
                ((x, y), (ma, mix), angle) = ellipse
                if ma/mix > 6 or ma/mix < 0.6:
                    continue
                cv2.ellipse(image, ellipse,(0, 0, 255),2)
            except:
                pass
            
            
            #checking score
            #320 316
            #537 209
            
            dist_square = (position_of_basket[0] - center[0])**2 + (position_of_basket[1] - center[1])**2
            score = dist_square < radius_of_basket**2
            if score and recently_scored == 0:
                print("score")
                recently_scored = 30
            elif recently_scored != 0 and not score:
                recently_scored -= 1
            
            cv2.circle(image,center,radius,(255,0,0),2)
            
            cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
            #cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
            #    0.5, (255, 255, 255), 2)
 
        # show the output image
    cv2.circle(image, position_of_basket, 22, (255, 225, 0), 2)
    cv2.imshow("Image", image)
    #    cv2.waitKey(0)
        
    key = cv2.waitKey(1) & 0xFF
 
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break 
vs.release()
cv2.destroyAllWindows()
# radii = set(radii)
# radii
# no 2 baskets within 1 sec
# collect data from different angles
# injury detection

score
